# Checking enviroment states and action sets

In [1]:
import sys
sys.path.append("..")
from SARSA_Agent import *
from ENV_DETAILS import *

2023-04-03 10:30:38.722676: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
ENV = "CartPole-v0"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:
env = gym.make(ENV)#,new_step_api=True
env

<TimeLimit<OrderEnforcing<PassiveEnvChecker<CartPoleEnv<CartPole-v0>>>>>

In [4]:
s = env.observation_space.sample()
s, s.shape

(array([-1.0458909e+00,  1.6361862e+38, -2.4187055e-01, -1.0400335e+38],
       dtype=float32),
 (4,))

In [5]:
env.observation_space, env.observation_space.shape

(Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32),
 (4,))

In [6]:
s = env.reset()
env.action_space.n, env.action_space, env.step(1)

(2,
 Discrete(2),
 (array([ 0.00522471,  0.17668733,  0.00410144, -0.31918937], dtype=float32),
  1.0,
  False,
  False,
  {}))

### Training/Hyperparam run

In [7]:
%tensorboard --logdir logs_general/hyper

UsageError: Line magic function `%tensorboard` not found.


In [8]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_SARSA/"

In [9]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_SARSA"
    project_name = "keras_tunning_epsilon"
    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/SARSA_epsilon/", exploration_tech ='epsilon' ,
                         sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value = SUCESS_CRITERIA_VALUE,
                  end_of_episode = EPISODES, batch = 32,  evaluation_epoch = 2000, 
                   training_steps = 1000000, 
                   time_to_update_min = 100, time_to_update_max = 800,
                   lr_min = 0.000005, lr_max = 0.005,
                   discount_min = 0.90, discount_max = 0.99,
                   dense_min = 32, dense_max = 512,
                   environment_name = ENV ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 50,
            # distribution_strategy=tf.distribute.MirroredStrategy(),
            directory= dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])

    # project_name = "keras_tunning_boltzman"
    # tuner = kt.BayesianOptimization(
    #         MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyperV2/SARSA_boltzman/", exploration_tech ='boltzman' ),
    #         objective= kt.Objective('total_train_reward', direction="max"), 
    #         max_trials = 20,
    #         # distribution_strategy=tf.distribute.MirroredStrategy(),
    #         directory= dir,
    #         project_name=project_name
    #     )
    # tuner.search(x=[0], y=[1])
   
else : 
    print("Acquiring parameters ....")
    learning_rate = 0.1
    training_steps = 5000000
    exploration_tech = "epsilon"
    discount = 0.99
    time_to_update = 100
    dense_units = 32
    end_of_episode = EPISODES
    sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH
    sucess_criteria_value = SUCESS_CRITERIA_VALUE
    run_training(training_steps, learning_rate,  exploration_tech, discount, time_to_update, dense_units, writer, end_of_episode, sucess_criteria_epochs, sucess_criteria_value)

Trial 50 Complete [01h 10m 00s]
total_train_reward: 9.37

Best total_train_reward So Far: 303.45
Total elapsed time: 1d 00h 49m 52s
INFO:tensorflow:Oracle triggered exit


In [10]:
exploration_tech="epsilon"

hyperparam_combination =[]
for trials in tuner.oracle.get_best_trials(num_trials=3):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

    disc = trials.hyperparameters.values["discount"]
    end_ep = 1000
    train_steps = 5000000
    d = trials.hyperparameters.values["dense_units"]
    lr = trials.hyperparameters.values["learning_rate"]
    time_to_update = trials.hyperparameters.values["time_to_update"]

    # hyperparam_combination.append((disc, end_ep, lr, tau_update_network, exploration_tech, train_steps, d, time_to_update))


Trial id :18 | Score :303.45 --> {'discount': 0.92, 'learning_rate': 0.0023892089991202374, 'time_to_update': 200, 'dense_units': 493}
Trial id :29 | Score :275.1 --> {'discount': 0.9, 'learning_rate': 0.0033187624977281524, 'time_to_update': 200, 'dense_units': 457}
Trial id :24 | Score :153.72 --> {'discount': 0.92, 'learning_rate': 0.0034375623201747746, 'time_to_update': 100, 'dense_units': 508}


In [11]:
len(hyperparam_combination), hyperparam_combination

(0, [])

In [12]:
# run_hyperparam(total_files_number = 4, TUNING_TYPE= "MANUAL",  TUNING_TYPE = TUNING_TYPE, hyperparam_combination = hyperparam_combination)

In [13]:
val_env = gym.make(ENV , render_mode = "rgb_array")
dir = r"Hyperparam_kt_SARSA"

for exploration_tech in ["epsilon", "boltzman"]:

    project_name = "keras_tunning_"+exploration_tech
    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/SARSA_"+exploration_tech+"/", exploration_tech =exploration_tech,
                         sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value = SUCESS_CRITERIA_VALUE,
                  end_of_episode = EPISODES, batch = 32,  evaluation_epoch = 2000, 
                   training_steps = 1000000, 
                   time_to_update_min = 100, time_to_update_max = 800,
                   lr_min = 0.000005, lr_max = 0.005,
                   discount_min = 0.90, discount_max = 0.99,
                   dense_min = 32, dense_max = 512,
                   environment_name = ENV ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 50,
            # distribution_strategy=tf.distribute.MirroredStrategy(),
            directory= dir,
            project_name=project_name
        )

    for trials in tuner.oracle.get_best_trials(num_trials=1):
        print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
        

    env_model = tuner.get_best_models()[0]
    final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./SARSA_"+exploration_tech+"_video.mp4")
    print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))

    

INFO:tensorflow:Reloading Tuner from Hyperparam_kt_SARSA/keras_tunning_epsilon/tuner0.json
Trial id :18 | Score :303.45 --> {'discount': 0.92, 'learning_rate': 0.0023892089991202374, 'time_to_update': 200, 'dense_units': 493}
Trial number :  50
Moviepy - Building video ./SARSA_epsilon_video.mp4.
Moviepy - Writing video ./SARSA_epsilon_video.mp4



Moviepy - Done !
Moviepy - video ready ./SARSA_epsilon_video.mp4
Final mean reward ' epsilon ': 166.08
INFO:tensorflow:Reloading Tuner from Hyperparam_kt_SARSA/keras_tunning_boltzman/tuner0.json
Trial id :34 | Score :496.52 --> {'discount': 0.93, 'learning_rate': 0.0038001679575366872, 'time_to_update': 300, 'dense_units': 151}
Trial number :  50
Moviepy - Building video ./SARSA_boltzman_video.mp4.
Moviepy - Writing video ./SARSA_boltzman_video.mp4



Moviepy - Done !
Moviepy - video ready ./SARSA_boltzman_video.mp4
Final mean reward ' boltzman ': 200.0
